# 1. SEP28 PREPARATION

In [6]:
!unzip sep28_all_together

unzip:  cannot find or open sep28_all_together, sep28_all_together.zip or sep28_all_together.ZIP.


In [1]:
!ls ./sep28_all_together/

StutterTalk_43_55.wav	     WomenWhoStutter_88_92.wav
StutterTalk_43_56.wav	     WomenWhoStutter_88_93.wav
StutterTalk_43_57.wav	     WomenWhoStutter_88_94.wav
StutterTalk_43_58.wav	     WomenWhoStutter_88_95.wav
StutterTalk_43_59.wav	     WomenWhoStutter_88_96.wav
StutterTalk_4_35.wav	     WomenWhoStutter_88_97.wav
StutterTalk_43_5.wav	     WomenWhoStutter_88_98.wav
StutterTalk_43_60.wav	     WomenWhoStutter_88_99.wav
StutterTalk_43_61.wav	     WomenWhoStutter_88_9.wav
StutterTalk_43_62.wav	     WomenWhoStutter_8_8.wav
StutterTalk_43_63.wav	     WomenWhoStutter_89_0.wav
StutterTalk_43_64.wav	     WomenWhoStutter_89_10.wav
StutterTalk_43_65.wav	     WomenWhoStutter_89_11.wav
StutterTalk_43_66.wav	     WomenWhoStutter_89_12.wav
StutterTalk_43_67.wav	     WomenWhoStutter_89_13.wav
StutterTalk_43_68.wav	     WomenWhoStutter_89_14.wav
StutterTalk_43_69.wav	     WomenWhoStutter_89_15.wav
StutterTalk_4_36.wav	     WomenWhoStutter_89_16.wav
StutterTalk_43_6.wav	     WomenWhoStutter_89_17.wav
S

In [2]:
import os

root_folder = './sep28_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 21856


In [3]:
import pandas as pd

df_sep = pd.read_csv('SEP_28k_labels.csv')
df_sep.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [18]:
print(df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['EpId'] == 99) & (df_sep['ClipId'] == 99)]['NoStutteredWords'])

27324    2
Name: NoStutteredWords, dtype: int64


In [4]:
tmp = df_sep[(df_sep['NoStutteredWords'] == 3) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] == 0)
                        & (df_sep['Block'] == 0)
                        & (df_sep['SoundRep'] == 0)    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0)]

len(df_sep), len(tmp)

(28177, 2479)

In [21]:
col_list_nostutter = [ 'Unsure', 
            'PoorAudioQuality',
            'WordRep', 
            'Block', 
            'SoundRep',
            'NoStutteredWords',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['Show'] == 'WomenWhoStutter') & (df_sep['Prolongation'] != 0) & (df_sep[col_list_nostutter].sum(axis=1) == 0)]

len(df_sep), len(tmp)

(28177, 61)

### Creating no stutter files with confidence 3 and count of 1698

In [4]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_imbalanced_6'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[8] == 3 and sum(features) == 3:
        print(show_name,pod_id,clip_id, features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)]['NoStutteredWords'])
        count += 1
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

HeStutters 18 115 [0 0 0 0 0 0 0 0 3 0 0 0] 2495    3
Name: NoStutteredWords, dtype: int64
HeStutters 3 28 [0 0 0 0 0 0 0 0 3 0 0 0] 557    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 8 13 [0 0 0 0 0 0 0 0 3 0 0 0] 20089    3
Name: NoStutteredWords, dtype: int64
MyStutteringLife 36 2 [0 0 0 0 0 0 0 0 3 0 0 0] 7471    3
Name: NoStutteredWords, dtype: int64
StutterTalk 33 2 [0 0 0 0 0 0 0 0 3 0 0 0] 11903    3
Name: NoStutteredWords, dtype: int64
StutterTalk 50 26 [0 0 0 0 0 0 0 0 3 0 0 0] 13105    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 4 36 [0 0 0 0 0 0 0 0 3 0 0 0] 19781    3
Name: NoStutteredWords, dtype: int64
HeStutters 20 32 [0 0 0 0 0 0 0 0 3 0 0 0] 2812    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 92 38 [0 0 0 0 0 0 0 0 3 0 0 0] 26666    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 90 1 [0 0 0 0 0 0 0 0 3 0 0 0] 26549    3
Name: NoStutteredWords, dtype: int64
WomenWhoStutter 96 52 [0 0 0 0 0 0 0 0 3 0 0 0] 26840    3
Name: NoStutt

In [5]:
import os

root_folder = './sep28_imbalanced_6/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 1698


### Creating stutter files for 'Prolongation' to be detected with confidence almost 3 and count of 118.

In [7]:
# tmp = df_sep[(df_sep['NoStutteredWords'] == 0) 
#                         & (df_sep['Unsure'] == 0)
#                         & (df_sep['PoorAudioQuality'] == 0)
#                         # & (df_sep['Prolongation'] == 0)
#                         & (df_sep['Block'] == 0)
#                         & ((df_sep['SoundRep'] >= 2) | (df_sep['Interjection'] >= 2) | ((df_sep['Prolongation'] == 0))    
#                         # & (df_sep['WordRep'] >= 2)
#                         & (df_sep['DifficultToUnderstand'] == 0)
#                         # & (df_sep['Interjection'] == 0)
#                         & (df_sep['NaturalPause'] == 0)
#                         & (df_sep['Music'] == 0)
#                         & (df_sep['NoSpeech'] == 0)]

tmp = df_sep[((df_sep['NoStutteredWords'] == 0) 
                        & (df_sep['Unsure'] == 0)
                        & (df_sep['PoorAudioQuality'] == 0)
                        & (df_sep['Prolongation'] != 0)
                        & (df_sep['Block'] == 0)
                        & ((df_sep['SoundRep'] == 0))    
                        & (df_sep['WordRep'] == 0)
                        & (df_sep['DifficultToUnderstand'] == 0)
                        & (df_sep['Interjection'] == 0)
                        & (df_sep['NaturalPause'] == 0)
                        & (df_sep['Music'] == 0)
                        & (df_sep['NoSpeech'] == 0))]

len(df_sep), len(tmp)

(28177, 172)

In [8]:
col_list_stutter = [ 'Unsure', 
            'PoorAudioQuality',
            'Prolongation', 
            'Block', 
            'SoundRep',
            'WordRep',
            'DifficultToUnderstand',
            'Interjection',
            'NaturalPause',
            'Music',
            'NoSpeech']

tmp = df_sep[(df_sep['NoStutteredWords'] == 0) & (df_sep[col_list_stutter].sum(axis=1) == 3)]

len(df_sep), len(tmp)

(28177, 1295)

In [8]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './sep28_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './sep28_imbalanced_6'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if (features[2] != 0 and sum(features) == features[2]):
        print(show_name,pod_id,clip_id)
        print(features,df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)][['Prolongation','Block','SoundRep','WordRep']])

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


WomenWhoStutter 50 372
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
23566             3      0         0        0
WomenWhoStutter 88 63
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
26345             3      0         0        0
StutterTalk 64 75
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
13849             3      0         0        0
StutterTalk 64 27
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
13801             3      0         0        0
WomenWhoStutter 13 36
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
20472             3      0         0        0
WomenWhoStutter 2 32
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
19523             3      0         0        0
WomenWhoStutter 30 77
[0 0 3 0 0 0 0 0 0 0 0 0]        Prolongation  Block  SoundRep  WordRep
21700             3      0         0        0
MyStutteringLife 34 16
[0 0 

In [9]:
import os

root_folder = './sep28_imbalanced_6/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)


Total number of files: 1816


### Renaming files in sep28_balanced_4 to nostutter (1698) and stutter files(118):

In [10]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './sep28_imbalanced_6/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_sep[(df_sep['Show'] == show_name) & (df_sep['EpId'] == pod_id) & (df_sep['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] == 3:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    elif features[2] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0

In [11]:
import os

# Specify the directory where your .wav files are located
directory = './sep28_imbalanced_6/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 1698 , 118 and tot 1816


In [12]:
!ls ./sep28_imbalanced_6/

nostutter_HeStutters_0_11.wav	       nostutter_StutterTalk_6_90.wav
nostutter_HeStutters_0_12.wav	       nostutter_StutterTalk_69_11.wav
nostutter_HeStutters_0_15.wav	       nostutter_StutterTalk_69_13.wav
nostutter_HeStutters_0_16.wav	       nostutter_StutterTalk_69_17.wav
nostutter_HeStutters_0_20.wav	       nostutter_StutterTalk_69_18.wav
nostutter_HeStutters_0_22.wav	       nostutter_StutterTalk_69_23.wav
nostutter_HeStutters_0_24.wav	       nostutter_StutterTalk_69_24.wav
nostutter_HeStutters_0_26.wav	       nostutter_StutterTalk_69_2.wav
nostutter_HeStutters_0_29.wav	       nostutter_StutterTalk_69_31.wav
nostutter_HeStutters_0_2.wav	       nostutter_StutterTalk_69_34.wav
nostutter_HeStutters_0_32.wav	       nostutter_StutterTalk_69_38.wav
nostutter_HeStutters_0_34.wav	       nostutter_StutterTalk_69_7.wav
nostutter_HeStutters_0_35.wav	       nostutter_StutterTalk_6_99.wav
nostutter_HeStutters_0_36.wav	       nostutter_StutterTalk_70_13.wav
nostutter_HeStutters_0_4.wav	       nos

### Splitting to train, val, test sets

In [13]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './sep28_imbalanced_6'

# Specify the destination directories for the split files
destination_directory_1 = './train_sep28_imbalanced_6'
destination_directory_2 = './val_sep28_imbalanced_6'
destination_directory_3 = './test_sep28_imbalanced_6'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)


1814


In [14]:
# Specify the directory where your .wav files are located
directory1 = './train_sep28_imbalanced_6'
directory2 = './val_sep28_imbalanced_6'
directory3 = './test_sep28_imbalanced_6'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

**********Sep28 dataset**********
Number of nostutter files in the train folder: 1367 and tot is 1452
Number of nostutter files in the val folder: 162 and tot is 181
Number of nostutter files in the test folder: 167 and tot is 181


In [ ]:
# import random
# import shutil
# import os
# import tensorflow as tf

# # Specify the source directory where your files are located
# source_directory = './fluency_bank_all_together'

# # Specify the destination directories for the split files
# destination_directory_1 = './fluency_balanced'
# destination_directory_2 = './fluency_inference'

# # Create the destination directories if they don't exist
# for directory in [destination_directory_1, destination_directory_2]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# # Shuffle the files in the source directory
# files = os.listdir(source_directory)
# random.shuffle(files)

# # Calculate the number of files for each split
# total_files = len(files)
# split_counts = [800, total_files-800]

# # Split the files into different lists based on the split counts
# file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]]]


# def check_elements(lst, positions):
#     non_zero_count = 0

#     for i in range(len(lst)):
#         if i in positions:
#             if i == 0 and lst[i] == 3:
#                 non_zero_count += 1
#             elif i != 0 and lst[i] >= 0:
#                 non_zero_count += 1
#         elif lst[i] != 0:
#             return False
#     return  non_zero_count >= 1

# specific_positions = [8, 9, 10, 11]

# count = 0
# # Move the files to the destination directories
# for i, file_split in enumerate(file_splits):
#     if i == 0:
#         destination_directory = destination_directory_1
#     else:
#         destination_directory = destination_directory_2

#     for filename in file_split:

#         path_parts = tf.strings.split(filename, '/')
#         path_end = path_parts[-1]
#         file_parts = tf.strings.split(path_end, '_')
#         show_name = file_parts[0].numpy().decode('utf-8')
#         pod_id = int(file_parts[-2])
#         clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
#         features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

#         if check_elements(features, specific_positions):
#             print(show_name,pod_id,clip_id)
#             print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
                


#             # source_path = os.path.join(source_directory, filename)
#             # destination_path = os.path.join(destination_directory, filename)
#             # shutil.copy2(source_path, destination_path)
    
#             count += 1

# print(count)

# 2. FLUENCY BANK PREPARATION

In [ ]:
!unzip ./fluency_bank_all_together.zip

In [14]:
!ls ./fluency_bank_all_together

FluencyBank_024_111.wav  FluencyBank_096_84.wav   FluencyBank_173_56.wav
FluencyBank_024_112.wav  FluencyBank_096_85.wav   FluencyBank_173_57.wav
FluencyBank_024_113.wav  FluencyBank_096_86.wav   FluencyBank_173_58.wav
FluencyBank_024_114.wav  FluencyBank_096_87.wav   FluencyBank_173_59.wav
FluencyBank_024_115.wav  FluencyBank_096_88.wav   FluencyBank_173_5.wav
FluencyBank_024_116.wav  FluencyBank_096_89.wav   FluencyBank_173_60.wav
FluencyBank_024_117.wav  FluencyBank_096_8.wav	  FluencyBank_173_61.wav
FluencyBank_024_118.wav  FluencyBank_096_90.wav   FluencyBank_173_62.wav
FluencyBank_024_119.wav  FluencyBank_096_91.wav   FluencyBank_173_63.wav
FluencyBank_024_11.wav	 FluencyBank_096_92.wav   FluencyBank_173_64.wav
FluencyBank_024_120.wav  FluencyBank_096_93.wav   FluencyBank_173_65.wav
FluencyBank_024_121.wav  FluencyBank_096_94.wav   FluencyBank_173_66.wav
FluencyBank_024_122.wav  FluencyBank_096_9.wav	  FluencyBank_173_67.wav
FluencyBank_024_123.wav  FluencyBank_097_0.wav	  Fluenc

In [15]:
import os

root_folder = './fluency_bank_all_together/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 4144


In [20]:
import pandas as pd

df_flu = pd.read_csv('fluencybank_labels.csv')
df_flu.head()

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,FluencyBank,10,0,88960,136960,0,0,0,1,0,0,0,0,2,0,0,0
1,FluencyBank,10,1,1271520,1319520,0,0,0,0,0,0,0,0,3,0,0,0
2,FluencyBank,10,2,1813760,1861760,0,0,1,0,0,0,0,0,2,0,0,0
3,FluencyBank,10,3,1842720,1890720,0,0,1,0,0,0,0,0,2,1,0,0
4,FluencyBank,10,4,1893280,1941280,0,0,0,0,0,3,0,0,0,0,0,0


In [4]:
column_location = df_flu.columns.get_loc('NoStutteredWords')
loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

loc_NoStutteredWords,loc_NaturalPause,loc_Music,loc_NoSpeech

(13, 14, 15, 16)

In [5]:
listttt = df_flu[(df_flu['Show'] == 'FluencyBank') & (df_flu['EpId'] == 10) & (df_flu['ClipId'] == 0)].to_numpy()[0]
ffff = [0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0]

loc_NoStutteredWords = df_flu.columns.get_loc('NoStutteredWords')
loc_NaturalPause = df_flu.columns.get_loc('NaturalPause')
loc_Music = df_flu.columns.get_loc('Music')
loc_NoSpeech = df_flu.columns.get_loc('NoSpeech')

def check_elements(lst, positions):
    non_zero_count = 0

    for i in range(len(lst)):
        if i in positions:
            if i == 0 and lst[i] == 3:
                non_zero_count += 1
            elif i != 0 and lst[i] >= 0:
                non_zero_count += 1
        elif lst[i] != 0:
            return False
    return  non_zero_count >= 1

specific_positions = [8, 9, 10, 11]

check_elements(ffff, specific_positions), listttt[5:]

(True, array([0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0], dtype=object))

### Creating no stutter files with confidence 3 and count of 643 

In [22]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 3) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] == 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] == 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 643)

In [17]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_imbalanced_6'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if features[8] == 3 and sum(features) == 3:
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['NoStutteredWords','NaturalPause','Music','NoSpeech']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)

print(count)

[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
274                 3             0      0         0
FluencyBank 99 153
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
2252                 3             0      0         0
FluencyBank 230 146
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3916                 3             0      0         0
FluencyBank 218 37
[0 0 0 0 0 0 0 0 3 0 0 0]       NoStutteredWords  NaturalPause  Music  NoSpeech
3593                 3             0      0         0
FluencyBank 24 127
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
497                 3             0      0         0
FluencyBank 24 179
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
549                 3             0      0         0
FluencyBank 46 85
[0 0 0 0 0 0 0 0 3 0 0 0]      NoStutteredWords  NaturalPause  Music  NoSpeech
734      

In [19]:
import os

root_folder = './fluency_imbalanced_6/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 643


### Creating stutter files for 'Prolongation' to be detected as stuttering situation with confidence almost 3 and count of 47.

In [21]:
tmp = df_flu[(df_flu['NoStutteredWords'] == 0) 
                        & (df_flu['Unsure'] == 0)
                        & (df_flu['PoorAudioQuality'] == 0)
                        & (df_flu['Prolongation'] != 0)
                        & (df_flu['Block'] == 0)
                        & (df_flu['SoundRep'] == 0)    
                        & (df_flu['WordRep'] == 0)
                        & (df_flu['DifficultToUnderstand'] == 0)
                        & (df_flu['Interjection'] == 0)
                        & (df_flu['NaturalPause'] == 0)
                        & (df_flu['Music'] == 0)
                        & (df_flu['NoSpeech'] == 0)]

len(df_flu), len(tmp)

(4144, 47)

In [17]:
loc_Prolongation = df_flu.columns.get_loc('Prolongation')
loc_Block = df_flu.columns.get_loc('Block')
loc_SoundRep = df_flu.columns.get_loc('SoundRep')
loc_WordRep = df_flu.columns.get_loc('WordRep')
loc_Interjection = df_flu.columns.get_loc('Interjection')

loc_Prolongation, loc_Block, loc_SoundRep, loc_WordRep, loc_Interjection

(7, 8, 9, 10, 12)

In [22]:
import os
import random
import shutil
import tensorflow as tf

# Specify the source directory where your files are located
source_directory = './fluency_bank_all_together/'
file_names = os.listdir(source_directory)
random.shuffle(file_names)

# Specify the destination directories for the split files
destination_directory = './fluency_imbalanced_6'

# Create the destination directories if they don't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)


count = 0

for filename in file_names:

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features =  df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]
    
    if ((features[2] != 0 and sum(features) == features[2])):
        print(show_name,pod_id,clip_id)
        print(features,df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)][['Prolongation', 'Block','SoundRep','WordRep']])
        count += 1

        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)


print(count)

FluencyBank 99 44
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
2143             3      0         0        0
FluencyBank 24 76
[0 0 3 0 0 0 0 0 0 0 0 0]      Prolongation  Block  SoundRep  WordRep
446             3      0         0        0
FluencyBank 97 174
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
1952             3      0         0        0
FluencyBank 93 56
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
1646             3      0         0        0
FluencyBank 178 41
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
3216             3      0         0        0
FluencyBank 81 35
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation  Block  SoundRep  WordRep
1203             3      0         0        0
FluencyBank 46 52
[0 0 3 0 0 0 0 0 0 0 0 0]      Prolongation  Block  SoundRep  WordRep
701             3      0         0        0
FluencyBank 178 114
[0 0 3 0 0 0 0 0 0 0 0 0]       Prolongation

In [23]:
import os

root_folder = './fluency_imbalanced_6/'  # Replace with the actual path to your root folder

total_files = 0

# Walk through the root folder and its subfolders
for dirpath, dirnames, filenames in os.walk(root_folder):
    total_files += len(filenames)

print("Total number of files:", total_files)

Total number of files: 690


### Renaming files in fluency_balanced to nostutter (643) and stutter files(47):

In [24]:
import os
import tensorflow as tf

# Specify the source directory where your files are located
folder_path = './fluency_imbalanced_6/'

for filename in os.listdir(folder_path):

    path_parts = tf.strings.split(filename, '/')
    path_end = path_parts[-1]
    file_parts = tf.strings.split(path_end, '_')
    show_name = file_parts[0].numpy().decode('utf-8')
    pod_id = int(file_parts[-2])
    clip_id = int(tf.strings.split(file_parts[-1], '.')[-2])
    features = df_flu[(df_flu['Show'] == show_name) & (df_flu['EpId'] == pod_id) & (df_flu['ClipId'] == clip_id)].to_numpy()[0][5:]

    
    if features[8] == 3:
        print(features)
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "nostutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

    elif features[2] != 0:
        old_file_path = os.path.join(folder_path, filename)
        new_filename = "stutter_" + filename  # Replace "new_name_" with your desired naming convention
        new_file_path = os.path.join(folder_path, new_filename)
        os.rename(old_file_path, new_file_path)

[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0 0 0 3 0 0 0]
[0 0 0 0 0 0

In [25]:
import os

# Specify the directory where your .wav files are located
directory = './fluency_imbalanced_6/'

# Initialize a counter variable
count = 0
count1 = 0
# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.startswith('nostutter'):
        count += 1
    if filename.startswith('stutter'):
        count1 += 1

print(f"Number of nostutter and stutter files: {count} , {count1} and tot {count+count1}")

Number of nostutter and stutter files: 643 , 47 and tot 690


In [26]:
!ls ./fluency_imbalanced_6

nostutter_FluencyBank_010_13.wav   nostutter_FluencyBank_099_180.wav
nostutter_FluencyBank_010_1.wav    nostutter_FluencyBank_099_181.wav
nostutter_FluencyBank_010_20.wav   nostutter_FluencyBank_099_22.wav
nostutter_FluencyBank_010_21.wav   nostutter_FluencyBank_099_25.wav
nostutter_FluencyBank_010_30.wav   nostutter_FluencyBank_099_26.wav
nostutter_FluencyBank_010_31.wav   nostutter_FluencyBank_099_31.wav
nostutter_FluencyBank_010_32.wav   nostutter_FluencyBank_099_38.wav
nostutter_FluencyBank_010_34.wav   nostutter_FluencyBank_099_40.wav
nostutter_FluencyBank_010_37.wav   nostutter_FluencyBank_099_46.wav
nostutter_FluencyBank_010_42.wav   nostutter_FluencyBank_099_52.wav
nostutter_FluencyBank_010_46.wav   nostutter_FluencyBank_099_68.wav
nostutter_FluencyBank_010_48.wav   nostutter_FluencyBank_099_88.wav
nostutter_FluencyBank_010_51.wav   nostutter_FluencyBank_099_90.wav
nostutter_FluencyBank_010_53.wav   nostutter_FluencyBank_099_93.wav
nostutter_FluencyBank_010_56.wav   nostutter_F

### Splitting to train, val, test sets

In [5]:
import random
import shutil
import os

# Specify the source directory where your files are located
source_directory = './fluency_balanced_3'

# Specify the destination directories for the split files
destination_directory_1 = './train_fluency_3'
destination_directory_2 = './val_fluency_3'
destination_directory_3 = './test_fluency_3'

# Create the destination directories if they don't exist
for directory in [destination_directory_1, destination_directory_2, destination_directory_3]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Shuffle the files in the source directory
files = os.listdir(source_directory)
random.shuffle(files)

# Define the sizes for each split
split_sizes = [0.8, 0.1, 0.1]

# Calculate the number of files for each split
total_files = len(files)
split_counts = [int(size * total_files) for size in split_sizes]

# Split the files into different lists based on the split counts
file_splits = [files[:split_counts[0]], files[split_counts[0]:split_counts[0] + split_counts[1]], files[-split_counts[2]:]]

count = 0
# Move the files to the destination directories
for i, file_split in enumerate(file_splits):
    if i == 0:
        destination_directory = destination_directory_1
    elif i == 1:
        destination_directory = destination_directory_2
    else:
        destination_directory = destination_directory_3

    for filename in file_split:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.copy2(source_path, destination_path)
    
        count += 1

print(count)

In [6]:
# Specify the directory where your .wav files are located
directory1 = './train_fluency_3'
directory2 = './val_fluency_3'
directory3 = './test_fluency_3'

# Initialize a counter variable
count_train = 0
count_val = 0
count_test = 0

count_train_tot = 0
count_val_tot = 0
count_test_tot = 0

# Iterate over the files in the directory
for filename in os.listdir(directory1):
    if filename.startswith('nostutter'):
        count_train += 1
    if filename.endswith('.wav'):
        count_train_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory2):
    if filename.startswith('nostutter'):
        count_val += 1
    if filename.endswith('.wav'):
        count_val_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

# Iterate over the files in the directory
for filename in os.listdir(directory3):
    if filename.startswith('nostutter'):
        count_test += 1    
    if filename.endswith('.wav'):
        count_test_tot += 1
    if not filename.endswith('.wav'):
        print('error')
    if not filename.startswith('nostutter') and not filename.startswith('stutter'):
        print(filename)

print('**********Sep28 dataset**********')
print(f"Number of nostutter files in the train folder: {count_train} and tot is {count_train_tot}")
print(f"Number of nostutter files in the val folder: {count_val} and tot is {count_val_tot}")
print(f"Number of nostutter files in the test folder: {count_test} and tot is {count_test_tot}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c74a7a1-d505-4b2b-a4fa-efea0c0f20da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>